In [1]:
cd ../.. 

/home/wtaisner/PycharmProjects/reasonable-embeddings


In [2]:
from pathlib import Path
base_dir = Path('local/out/elpp/exp1')

base_dir.mkdir(parents=True, exist_ok=True)

In [3]:
class RetardedCodeError(Exception):
    def __init__(self, message="Retarded code encountered"):
        self.message = message
        super().__init__(self.message)

In [4]:
from src.elpp.gen import gen
import numpy as np
from multiprocessing import Pool, cpu_count

rng = np.random.default_rng(seed=42)

n_concepts = 300
n_roles = 5
n_proofs = 100

n_epochs = 600


def generate_reasoner(_):
    reasoner = gen(n_concepts=n_concepts, n_roles=n_roles, rng=rng, n_proofs=n_proofs, n_epochs=n_epochs)
    if reasoner is None:
        raise RetardedCodeError()
    return reasoner

with Pool(cpu_count() // 2) as p:
    reasoners = p.map(generate_reasoner, range(40))

if None in reasoners:
    raise RetardedCodeError()

 92%|█████████▏| 554/600 [00:05<00:00, 107.22it/s]


In [5]:
None in reasoners

False

In [7]:
import lzma
import dill

with lzma.open(base_dir / f'reasoners_{n_concepts}_concepts_{n_roles}_roles_{n_proofs}_proofs.dill.xz', 'wb') as f:
    dill.dump(reasoners, f)

In [8]:
rng = np.random.default_rng(seed=0xc0ffee)

with Pool(cpu_count() // 2) as p:
    test_reasoners = p.map(generate_reasoner, range(20))

  0%|          | 0/600 [00:00<?, ?it/s]

 73%|███████▎  | 439/600 [00:04<00:01, 109.35it/s]


In [9]:
None in test_reasoners

False

In [10]:
with lzma.open(base_dir / f'test_reasoners_{n_concepts}_concepts_{n_roles}_roles_{n_proofs}_proofs.dill.xz', 'wb') as f:
    dill.dump(test_reasoners, f)